WARNING: this example is not yet fully functional
https://github.com/imperial-qore/line-solver/wiki/Getting-started#example-2-a-multiclass-mg1-queue

In [1]:
from line_solver import *
import os
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
model = Network('M/G/1')
source = Source(model, 'Source')
queue = Queue(model, 'Queue', SchedStrategy.FCFS)
sink = Sink(model, 'Sink')
jobclass1 = OpenClass(model, 'Class1')
jobclass2 = OpenClass(model, 'Class2')
source.setArrival(jobclass1, Exp(0.5))
source.setArrival(jobclass2, Exp(0.5))
queue.setService(jobclass1, Erlang.fitMeanAndSCV(1, 1 / 3))
queue.setService(jobclass2, Replayer("example_trace.txt"))


In [3]:
P = model.initRoutingMatrix()
P.set(jobclass1, jobclass1, Network.serialRouting(source,queue,sink))
P.set(jobclass2, jobclass2, Network.serialRouting(source,queue,sink))
model.link(P)

In [4]:
jmtAvgTable = SolverJMT(model,'seed',23000).getAvgTable()

JMT Model: /tmp/workspace/jsim/11840675272245223466/model.jsim
JMT Analysis completed. Runtime: 1.066 seconds
  Station JobClass      QLen      Util     RespT    ResidT      ArvR      Tput
0  Source   Class1  0.000000  0.000000  0.000000  0.000000  0.000000  0.500173
1  Source   Class2  0.000000  0.000000  0.000000  0.000000  0.000000  0.491145
2   Queue   Class1  0.861526  0.498399  1.738891  1.738891  0.500173  0.499527
3   Queue   Class2  0.437515  0.049184  0.858789  0.858789  0.491145  0.490639


In [5]:
queue.setService(jobclass2, Replayer("example_trace.txt").fitAPH())
model.reset()
# TODO: results are slightly different than in MATLAB as the latter uses also the skewness within fitAPH
jmtAvgTable = SolverJMT(model,'seed',23000).getAvgTable()

JMT Model: /tmp/workspace/jsim/15088268967390222599/model.jsim
JMT Analysis completed. Runtime: 0.787 seconds
  Station JobClass      QLen      Util     RespT    ResidT      ArvR      Tput
0  Source   Class1  0.000000  0.000000  0.000000  0.000000  0.000000  0.501685
1  Source   Class2  0.000000  0.000000  0.000000  0.000000  0.000000  0.506538
2   Queue   Class1  0.880661  0.494749  1.737321  1.737321  0.501685  0.506679
3   Queue   Class2  0.438433  0.053432  0.817414  0.817414  0.506538  0.507503


In [6]:
#ctmcAvgTable2 = SolverCTMC(model,'cutoff',2,'verbose',True).getAvgTable()
#ctmcAvgTable4 = SolverCTMC(model,'cutoff',4,'verbose',True).getAvgTable()